In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.\
        builder.\
        config("spark.ui.port","0").\
        master("yarn").\
        enableHiveSupport().\
        config("spark.sql.warehouse.dir", "/user/band/warehouse").\
        appName("Section14").\
        getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2021-12-28 08:45:32,689 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [19]:
spark.sql("DROP DATABASE IF EXISTS hr_db")

""


In [21]:
spark.sql("""
CREATE DATABASE IF NOT EXISTS hr_db
""")

2021-12-24 08:52:12,389 WARN metastore.ObjectStore: Failed to get database hr_db, returning NoSuchObjectException


""


In [22]:
spark.sql("SHOW DATABASES").show()

+-----------+
|  namespace|
+-----------+
|band_retail|
|    default|
|      hr_db|
|  ordertest|
|  retail_db|
+-----------+



In [23]:
spark.sql("USE hr_db")

""


In [24]:
spark.sql("""
    CREATE TABLE employees(
        employee_id INT,
        first_name VARCHAR(25),
        last_name VARCHAR(25),
        email VARCHAR(100),
        phone_number VARCHAR(20),
        hire_date DATE,
        job_id VARCHAR(10),
        salary DECIMAL(8,2),
        commission_pct DECIMAL(2,2),
        manager_id INT,
        department_id INT
    )ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t'
""")

2021-12-24 08:53:03,840 WARN metastore.HiveMetaStore: Location: hdfs://127.0.0.1:9000/user/band/warehouse/hr_db.db/employees specified for non-external table:employees


""


In [4]:
spark.sql("LOAD DATA LOCAL INPATH '/data/hr_db/employees/part-m-00000' INTO TABLE employees")

2021-12-28 08:46:14,991 WARN conf.HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
2021-12-28 08:46:15,005 WARN conf.HiveConf: HiveConf of name hive.stats.retries.wait does not exist
2021-12-28 08:46:35,544 WARN metastore.ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


""


In [5]:
spark.sql("SELECT * FROM employees LIMIT 10").show(truncate=False)

+-----------+----------+---------+--------+------------------+----------+------+--------+--------------+----------+-------------+
|employee_id|first_name|last_name|email   |phone_number      |hire_date |job_id|salary  |commission_pct|manager_id|department_id|
+-----------+----------+---------+--------+------------------+----------+------+--------+--------------+----------+-------------+
|155        |Oliver    |Tuvault  |OTUVAULT|011.44.1344.486508|1999-11-23|SA_REP|7000.00 |0.15          |145       |80           |
|156        |Janette   |King     |JKING   |011.44.1345.429268|1996-01-30|SA_REP|10000.00|0.35          |146       |80           |
|157        |Patrick   |Sully    |PSULLY  |011.44.1345.929268|1996-03-04|SA_REP|9500.00 |0.35          |146       |80           |
|158        |Allan     |McEwen   |AMCEWEN |011.44.1345.829268|1996-08-01|SA_REP|9000.00 |0.35          |146       |80           |
|159        |Lindsey   |Smith    |LSMITH  |011.44.1345.729268|1997-03-10|SA_REP|8000.00 |0

In [36]:
spark.sql("""
    SELECT e.employee_id, e.department_id, e.salary, ae.department_salary_expense, ae.avg_salary_expense
    FROM employees AS e
    INNER JOIN (
        SELECT department_id, SUM(salary) AS department_salary_expense, AVG(salary) AS avg_salary_expense
        FROM employees
        GROUP BY department_id
    ) AS ae
    ON e.department_id = ae.department_id
    ORDER BY department_id, salary
""").show(10, truncate=False)

+-----------+-------------+--------+-------------------------+------------------+
|employee_id|department_id|salary  |department_salary_expense|avg_salary_expense|
+-----------+-------------+--------+-------------------------+------------------+
|200        |10           |4400.00 |4400.00                  |4400.000000       |
|202        |20           |6000.00 |19000.00                 |9500.000000       |
|201        |20           |13000.00|19000.00                 |9500.000000       |
|119        |30           |2500.00 |24900.00                 |4150.000000       |
|118        |30           |2600.00 |24900.00                 |4150.000000       |
|117        |30           |2800.00 |24900.00                 |4150.000000       |
|116        |30           |2900.00 |24900.00                 |4150.000000       |
|115        |30           |3100.00 |24900.00                 |4150.000000       |
|114        |30           |11000.00|24900.00                 |4150.000000       |
|203        |40 

In [46]:
spark.sql("""
    SELECT e.employee_id, e.department_id, e.salary,
    SUM(e.salary) OVER (PARTITION BY e.department_id) AS SUM,
    AVG(e.salary) OVER (PARTITION BY e.department_id) AS AVG,
    MIN(e.salary) OVER (PARTITION BY e.department_id) AS MIN,
    MAX(e.salary) OVER (PARTITION BY e.department_id) AS MAX,
    COUNT(e.salary) OVER (PARTITION BY e.department_id) AS COUNT
    FROM employees AS e
    ORDER BY e.department_id, e.salary
""").show(10, truncate=False)

+-----------+-------------+--------+--------+-----------+-------+--------+-----+
|employee_id|department_id|salary  |SUM     |AVG        |MIN    |MAX     |COUNT|
+-----------+-------------+--------+--------+-----------+-------+--------+-----+
|178        |null         |7000.00 |7000.00 |7000.000000|7000.00|7000.00 |1    |
|200        |10           |4400.00 |4400.00 |4400.000000|4400.00|4400.00 |1    |
|202        |20           |6000.00 |19000.00|9500.000000|6000.00|13000.00|2    |
|201        |20           |13000.00|19000.00|9500.000000|6000.00|13000.00|2    |
|119        |30           |2500.00 |24900.00|4150.000000|2500.00|11000.00|6    |
|118        |30           |2600.00 |24900.00|4150.000000|2500.00|11000.00|6    |
|117        |30           |2800.00 |24900.00|4150.000000|2500.00|11000.00|6    |
|116        |30           |2900.00 |24900.00|4150.000000|2500.00|11000.00|6    |
|115        |30           |3100.00 |24900.00|4150.000000|2500.00|11000.00|6    |
|114        |30           |1

In [83]:
spark.sql("""
    SELECT employee_id, 
    first_name, 
    hire_date, 
    salary, 
    manager_id, 
    department_id,
    LEAD(hire_date) OVER(ORDER BY hire_date DESC, manager_id DESC) AS sorted_date
    
    FROM employees
""").show(10, truncate=False)

2021-12-24 12:36:13,451 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2021-12-24 12:36:13,451 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----------+----------+----------+--------+----------+-------------+-----------+
|employee_id|first_name|hire_date |salary  |manager_id|department_id|sorted_date|
+-----------+----------+----------+--------+----------+-------------+-----------+
|173        |Sundita   |2000-04-21|6100.00 |148       |80           |2000-04-21 |
|167        |Amit      |2000-04-21|6200.00 |147       |80           |2000-03-24 |
|166        |Sundar    |2000-03-24|6400.00 |147       |80           |2000-03-08 |
|128        |Steven    |2000-03-08|2200.00 |120       |50           |2000-02-23 |
|165        |David     |2000-02-23|6800.00 |147       |80           |2000-02-06 |
|136        |Hazel     |2000-02-06|2200.00 |122       |50           |2000-02-03 |
|183        |Girard    |2000-02-03|2800.00 |120       |50           |2000-01-29 |
|149        |Eleni     |2000-01-29|10500.00|100       |80           |2000-01-24 |
|164        |Mattea    |2000-01-24|7200.00 |147       |80           |2000-01-13 |
|199        |Dou

2021-12-24 12:36:13,691 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [110]:
# LEAD() function
spark.sql("""
    SELECT employee_id, 
    first_name, 
    hire_date, 
    salary, 
    manager_id, 
    department_id,
    LEAD(hire_date,0) OVER(ORDER BY hire_date DESC) AS LEAD_date
    FROM employees
    ORDER BY hire_date
    LIMIT 10
""").show(truncate=False)

2021-12-24 14:36:50,058 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2021-12-24 14:36:50,058 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2021-12-24 14:36:50,219 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----------+----------+----------+--------+----------+-------------+----------+
|employee_id|first_name|hire_date |salary  |manager_id|department_id|LEAD_date |
+-----------+----------+----------+--------+----------+-------------+----------+
|100        |Steven    |1987-06-17|24000.00|null      |90           |1987-06-17|
|200        |Jennifer  |1987-09-17|4400.00 |101       |10           |1987-09-17|
|101        |Neena     |1989-09-21|17000.00|100       |90           |1989-09-21|
|103        |Alexander |1990-01-03|9000.00 |102       |60           |1990-01-03|
|104        |Bruce     |1991-05-21|6000.00 |103       |60           |1991-05-21|
|102        |Lex       |1993-01-13|17000.00|100       |90           |1993-01-13|
|205        |Shelley   |1994-06-07|12000.00|101       |110          |1994-06-07|
|203        |Susan     |1994-06-07|6500.00 |101       |40           |1994-06-07|
|204        |Hermann   |1994-06-07|10000.00|101       |70           |1994-06-07|
|206        |William   |1994

In [108]:
# LEAD() function
spark.sql("""
    SELECT employee_id, 
    first_name, 
    hire_date, 
    salary, 
    manager_id, 
    department_id,
    LEAD(hire_date) OVER(PARTITION BY manager_id ORDER BY hire_date DESC) AS LEAD_date
    FROM employees
    ORDER BY hire_date
    LIMIT 10
""").show(truncate=False)

+-----------+----------+----------+--------+----------+-------------+----------+
|employee_id|first_name|hire_date |salary  |manager_id|department_id|LEAD_date |
+-----------+----------+----------+--------+----------+-------------+----------+
|100        |Steven    |1987-06-17|24000.00|null      |90           |null      |
|200        |Jennifer  |1987-09-17|4400.00 |101       |10           |null      |
|101        |Neena     |1989-09-21|17000.00|100       |90           |null      |
|103        |Alexander |1990-01-03|9000.00 |102       |60           |null      |
|104        |Bruce     |1991-05-21|6000.00 |103       |60           |null      |
|102        |Lex       |1993-01-13|17000.00|100       |90           |1989-09-21|
|203        |Susan     |1994-06-07|6500.00 |101       |40           |1994-06-07|
|204        |Hermann   |1994-06-07|10000.00|101       |70           |1994-06-07|
|205        |Shelley   |1994-06-07|12000.00|101       |110          |1987-09-17|
|206        |William   |1994

In [97]:
# LAG() function
spark.sql("""
    SELECT employee_id, 
    first_name, 
    hire_date, 
    salary, 
    manager_id, 
    department_id,
    LAG(hire_date,2,TO_DATE('0-0-0','y-m-d')) OVER(ORDER BY hire_date DESC) AS LAG_date
    FROM employees
""").show(10, truncate=False)

2021-12-24 14:15:54,739 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2021-12-24 14:15:54,739 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2021-12-24 14:15:54,911 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----------+----------+----------+--------+----------+-------------+----------+
|employee_id|first_name|hire_date |salary  |manager_id|department_id|LAG_date  |
+-----------+----------+----------+--------+----------+-------------+----------+
|167        |Amit      |2000-04-21|6200.00 |147       |80           |null      |
|173        |Sundita   |2000-04-21|6100.00 |148       |80           |null      |
|166        |Sundar    |2000-03-24|6400.00 |147       |80           |2000-04-21|
|128        |Steven    |2000-03-08|2200.00 |120       |50           |2000-04-21|
|165        |David     |2000-02-23|6800.00 |147       |80           |2000-03-24|
|136        |Hazel     |2000-02-06|2200.00 |122       |50           |2000-03-08|
|183        |Girard    |2000-02-03|2800.00 |120       |50           |2000-02-23|
|149        |Eleni     |2000-01-29|10500.00|100       |80           |2000-02-06|
|164        |Mattea    |2000-01-24|7200.00 |147       |80           |2000-02-03|
|199        |Douglas   |2000

In [141]:
spark.sql("""
    SELECT employee_id, 
    first_name, 
    hire_date, 
    salary, 
    manager_id, 
    department_id,
    FIRST_VALUE(hire_date) OVER(ORDER BY hire_date) AS first_value,
    LAST_VALUE(hire_date) OVER(ORDER BY hire_date) AS last_value
    FROM employees
""").show(10, truncate=False)

2021-12-24 15:36:05,369 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2021-12-24 15:36:05,369 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2021-12-24 15:36:05,560 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----------+----------+----------+--------+----------+-------------+-----------+----------+
|employee_id|first_name|hire_date |salary  |manager_id|department_id|first_value|last_value|
+-----------+----------+----------+--------+----------+-------------+-----------+----------+
|100        |Steven    |1987-06-17|24000.00|null      |90           |1987-06-17 |1987-06-17|
|200        |Jennifer  |1987-09-17|4400.00 |101       |10           |1987-06-17 |1987-09-17|
|101        |Neena     |1989-09-21|17000.00|100       |90           |1987-06-17 |1989-09-21|
|103        |Alexander |1990-01-03|9000.00 |102       |60           |1987-06-17 |1990-01-03|
|104        |Bruce     |1991-05-21|6000.00 |103       |60           |1987-06-17 |1991-05-21|
|102        |Lex       |1993-01-13|17000.00|100       |90           |1987-06-17 |1993-01-13|
|203        |Susan     |1994-06-07|6500.00 |101       |40           |1987-06-17 |1994-06-07|
|204        |Hermann   |1994-06-07|10000.00|101       |70           |1

In [151]:
spark.sql("""
    SELECT employee_id, 
    first_name, 
    hire_date, 
    salary, 
    manager_id, 
    department_id,
    RANK() OVER(ORDER BY salary) AS rank_salary
    FROM employees
    ORDER BY hire_date
""").show(10, truncate=False)

2021-12-24 15:47:06,150 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2021-12-24 15:47:06,151 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2021-12-24 15:47:06,320 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----------+----------+----------+--------+----------+-------------+-----------+
|employee_id|first_name|hire_date |salary  |manager_id|department_id|rank_salary|
+-----------+----------+----------+--------+----------+-------------+-----------+
|100        |Steven    |1987-06-17|24000.00|null      |90           |107        |
|200        |Jennifer  |1987-09-17|4400.00 |101       |10           |47         |
|101        |Neena     |1989-09-21|17000.00|100       |90           |105        |
|103        |Alexander |1990-01-03|9000.00 |102       |60           |81         |
|104        |Bruce     |1991-05-21|6000.00 |103       |60           |51         |
|102        |Lex       |1993-01-13|17000.00|100       |90           |105        |
|203        |Susan     |1994-06-07|6500.00 |101       |40           |57         |
|206        |William   |1994-06-07|8300.00 |205       |110          |77         |
|204        |Hermann   |1994-06-07|10000.00|101       |70           |89         |
|205        |She

In [172]:
spark.sql("""
    SELECT employee_id, 
    first_name, 
    hire_date, 
    salary, 
    manager_id, 
    department_id,
    DENSE_RANK() OVER(PARTITION BY manager_id ORDER BY salary DESC) AS rank_salary
    FROM employees
    ORDER BY manager_id ASC, salary DESC
    LIMIT 50
""").show(50, truncate=False)

+-----------+-----------+----------+--------+----------+-------------+-----------+
|employee_id|first_name |hire_date |salary  |manager_id|department_id|rank_salary|
+-----------+-----------+----------+--------+----------+-------------+-----------+
|100        |Steven     |1987-06-17|24000.00|null      |90           |1          |
|101        |Neena      |1989-09-21|17000.00|100       |90           |1          |
|102        |Lex        |1993-01-13|17000.00|100       |90           |1          |
|145        |John       |1996-10-01|14000.00|100       |80           |2          |
|146        |Karen      |1997-01-05|13500.00|100       |80           |3          |
|201        |Michael    |1996-02-17|13000.00|100       |20           |4          |
|147        |Alberto    |1997-03-10|12000.00|100       |80           |5          |
|114        |Den        |1994-12-07|11000.00|100       |30           |6          |
|148        |Gerald     |1999-10-15|11000.00|100       |80           |6          |
|149